## **`Carregamento dos dados GOOGLE MOBILITY e transformação em diferentes formatos`**

`Este notebook carrega os dados de https://www.google.com/covid19/mobility/ e constrói relatórios em diferentes formatos. `





In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import io
import os
import datetime

import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re
import json

import pandas as pd

In [0]:
def download_google_reports(
        directory_pdf=os.path.join(
            "/content/drive/My Drive/google_reports",
            "/content/drive/My Drive/pdf_reports"),
        directory_csv="/content/drive/My Drive/google_reports"):
    '''Download Google Community Mobility reports in CSV and PDF format

        Args:
            directory_pdf: directory to which PDF reports will be downloaded
            directory_csv: directory to which CSV report will be downloaded

        Returns:
            new_files (bool): flag indicating whether or not new files have been downloaded
    '''
    # get webpage source
    url = 'https://www.google.com/covid19/mobility/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    new_files = False

    # create directories if they don't exist
    if not os.path.exists(directory_pdf):
        os.makedirs(directory_pdf)
    if not os.path.exists(directory_csv):
        os.makedirs(directory_csv)

    # download CSV file
    csv_tag = soup.find('a', {"class": "icon-link"})
    link = csv_tag['href']
    file_name = "Global_Mobility_Report.csv"
    path = os.path.join(directory_csv, file_name)
    if not os.path.isfile(path):
        new_files = True
        urllib.request.urlretrieve(link, path)
        print(file_name)
        time.sleep(1)
    else:
        path_new = os.path.join(directory_csv, file_name + "_new")
        urllib.request.urlretrieve(link, path_new)
        if os.path.getsize(path) == os.path.getsize(path_new):
            os.remove(path_new)
        else:
            new_files = True
            os.remove(path)
            os.rename(path_new, path)
    # download PDFs
    json_data = re.search(
        r"window.templateData=JSON.parse\('([^']+)", response.text)
    json_data = bytes(json_data.groups()[0], 'utf-8').decode('unicode_escape')
    json_data = json.loads(json_data)

    def get_pdf_files(e):
        link = e['pdfLink']
        file_name = link[link.find('mobility') + len('mobility') + 1:]
        if link[-3:] == "pdf":
            path = os.path.join(directory_pdf, file_name)
            if not os.path.isfile(path):
                new_files = True
                urllib.request.urlretrieve(link, path)
                print(file_name)
                time.sleep(1)

    for elem in json_data['countries']:
        get_pdf_files(elem)
        for child in elem['childRegions']:
            get_pdf_files(child)

    if not new_files:
        print('Google: No updates')
    return new_files

In [0]:
def build_google_report(
        source="Global_Mobility_Report.csv",
        destination="mobility_report.csv",
        report_type="regions"):
    '''Build cleaned Google report for worldwide or for some country (currently only for the US)

        Args:
            source: location of the raw Google CSV report
            destination: destination file path
            report_type: two options available: "regions" - report for worldwide, "US" - report for the US
    '''
    df = pd.read_csv(source, low_memory=False)
    df = df.drop(columns=['country_region_code'])
    df = df.rename(
        columns={
            'country_region': 'country',
            'retail_and_recreation_percent_change_from_baseline': 'retail',
            'grocery_and_pharmacy_percent_change_from_baseline': 'grocery and pharmacy',
            'parks_percent_change_from_baseline': 'parks',
            'transit_stations_percent_change_from_baseline': 'transit stations',
            'workplaces_percent_change_from_baseline': 'workplaces',
            'residential_percent_change_from_baseline': 'residential'})
    if report_type == "regions":
        df = df[df['sub_region_2'].isnull()]
        df = df.drop(columns=['sub_region_2'])
        df = df.rename(columns={'sub_region_1': 'region'})
        df['region'].fillna('Total', inplace=True)
    elif report_type == "US":
        df = df[(df['country'] == "United States")]
        df = df.drop(columns=['country'])
        df = df.rename(
            columns={
                'sub_region_1': 'state',
                'sub_region_2': 'county'})
        df['state'].fillna('Total', inplace=True)
        df['county'].fillna('Total', inplace=True)
    df.to_csv(destination, index=False)

In [0]:
 def build_summary_report(
        google_source=os.path.join(
            "/content/drive/My Drive/google_reports",
            "Global_Mobility_Report.csv"),
    destination=os.path.join(
        "/content/drive/My Drive/summary_reports",
        "summary_report.csv")):
    '''Build a merged report from Google and Apple data

        Args:
            apple_source: location of the raw Apple CSV report
            google_source: location of the raw Google CSV report
            destination: destination file path
    '''

    # process google data
    google = pd.read_csv(google_source, low_memory=False)
    google['sub_region_1'].fillna('Total', inplace=True)
    google['sub_region_2'].fillna('Total', inplace=True)
    google = google.rename(
        columns={
            'country_region': 'country',
            'retail_and_recreation_percent_change_from_baseline': 'retail',
            'grocery_and_pharmacy_percent_change_from_baseline': 'grocery and pharmacy',
            'parks_percent_change_from_baseline': 'parks',
            'transit_stations_percent_change_from_baseline': 'transit stations',
            'workplaces_percent_change_from_baseline': 'workplaces',
            'residential_percent_change_from_baseline': 'residential'})
    
    summary = google.drop(
        columns=['country_region_code'])
    summary['sub_region_2'].fillna('Total', inplace=True)
    summary = summary.sort_values(
        by=['country', 'sub_region_1', 'sub_region_2', 'date'])
    summary.to_csv(destination, index=False)

In [0]:
def slice_summary_report(
    source=os.path.join(
        "/content/drive/My Drive/summary_reports",
        "summary_report.csv"),
        destination_regions=os.path.join(
            "/content/drive/My Drive/summary_reports",
            "summary_report_regions.csv"),
    destination_countries=os.path.join(
        "/content/drive/My Drive/summary_reports",
        "summary_report_countries.csv"),
    destination_US=os.path.join(
        "/content/drive/My Drive/summary_reports",
        "summary_report_PT.csv")):
   
    # read full summary report
    summary = pd.read_csv(source, low_memory=False)
    # create report #1
    regions = summary[summary['sub_region_2'] == 'Total']
    regions = regions.drop(columns=['sub_region_2'])
    regions.to_csv(destination_regions, index=False)
    # create report #2
    countries = summary[summary['sub_region_1'] == 'Total']
    countries = countries.drop(columns=['sub_region_1', 'sub_region_2'])
    countries.to_csv(destination_countries, index=False)
    # create report #3
    PT = summary[summary['country'] == 'Portugal']
    PT.to_csv(destination_US, index=False)

In [0]:
def csv_to_excel(csv_path, excel_path):
    """Helper function which create Excel file from CSV"""
    df = pd.read_csv(csv_path, low_memory=False)
    df.to_excel(excel_path, index=False, sheet_name='Data')

In [0]:
os.chdir('..')

In [10]:
new_files_status_google = download_google_reports()
if new_files_status_google:
    build_google_report(
        source=os.path.join(
            "/content/drive/My Drive/google_reports",
            "Global_Mobility_Report.csv"),
        destination=os.path.join(
            "/content/drive/My Drive/google_reports",
            "mobility_report_countries.csv"),
        report_type="regions")
    build_google_report(
        source=os.path.join(
            "/content/drive/My Drive/google_reports",
            "Global_Mobility_Report.csv"),
        destination=os.path.join(
            "/content/drive/My Drive/google_reports",
            "mobility_report_PT.csv"),
        report_type="PT")
    csv_to_excel(
        os.path.join(
            "/content/drive/My Drive/google_reports",
            "mobility_report_countries.csv"),
        os.path.join(
            "/content/drive/My Drive/google_reports",
            "mobility_report_countries.xlsx"))
    csv_to_excel(os.path.join("/content/drive/My Drive/google_reports", "mobility_report_PT.csv"),
                 os.path.join("/content/drive/My Drive/google_reports", "mobility_report_PT.xlsx"))

Global_Mobility_Report.csv


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: DeprecationWarning: invalid escape sequence '\/'


2020-05-07_AF_Mobility_Report_en.pdf
2020-05-07_AO_Mobility_Report_en.pdf
2020-05-07_AG_Mobility_Report_en.pdf
2020-05-07_AR_Mobility_Report_en.pdf
2020-05-07_AW_Mobility_Report_en.pdf
2020-05-07_AU_Mobility_Report_en.pdf
2020-05-07_AT_Mobility_Report_en.pdf
2020-05-07_BH_Mobility_Report_en.pdf
2020-05-07_BD_Mobility_Report_en.pdf
2020-05-07_BB_Mobility_Report_en.pdf
2020-05-07_BY_Mobility_Report_en.pdf
2020-05-07_BE_Mobility_Report_en.pdf
2020-05-07_BZ_Mobility_Report_en.pdf
2020-05-07_BJ_Mobility_Report_en.pdf
2020-05-07_BO_Mobility_Report_en.pdf
2020-05-07_BA_Mobility_Report_en.pdf
2020-05-07_BW_Mobility_Report_en.pdf
2020-05-07_BR_Mobility_Report_en.pdf
2020-05-07_BG_Mobility_Report_en.pdf
2020-05-07_BF_Mobility_Report_en.pdf
2020-05-07_KH_Mobility_Report_en.pdf
2020-05-07_CM_Mobility_Report_en.pdf
2020-05-07_CA_Mobility_Report_en.pdf
2020-05-07_CV_Mobility_Report_en.pdf
2020-05-07_CL_Mobility_Report_en.pdf
2020-05-07_CO_Mobility_Report_en.pdf
2020-05-07_CR_Mobility_Report_en.pdf
2

In [0]:
if new_files_status_google:
    build_summary_report()
    csv_to_excel(
        os.path.join(
            "/content/drive/My Drive/summary_reports",
            "summary_report.csv"),
        os.path.join(
            "/content/drive/My Drive/summary_reports",
            "summary_report.xlsx"))
    # slice summary report
    slice_summary_report()
    csv_to_excel(
        os.path.join(
            "/content/drive/My Drive/summary_reports",
            "summary_report_regions.csv"),
        os.path.join(
            "/content/drive/My Drive/summary_reports",
            "summary_report_regions.xlsx"))
    csv_to_excel(
        os.path.join(
            "/content/drive/My Drive/summary_reports",
            "summary_report_countries.csv"),
        os.path.join(
            "/content/drive/My Drive/summary_reports",
            "summary_report_countries.xlsx"))
    csv_to_excel(
        os.path.join(
            "/content/drive/My Drive/summary_reports",
            "summary_report_PT.csv"),
        os.path.join(
            "/content/drive/My Drive/summary_reports",
            "summary_report_PT.xlsx"))